In [148]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import requests

In [149]:
df = pd.read_csv('exact.csv')
df.head(1)

,Id,VlrNota,Dt. Evento,Ano Evento,Receita,Dt. Entrada,Tempo de vida da safra,Ano Entrada,Porte Ajustado,Origem ajustada,Integração RD Mkt e CRM,Webphone,Exact Club,Média de Leads em Entrada (Categorias),CNAE
0,1,"R$ 597,00",01/10/2022,2022,Spotter,01/08/2020,27,2020,Small,Lista ativa,False,True,False,-,INTERNACIONAL


In [150]:
ch = pd.read_csv('chave.csv')
ch.head(1)

,ID,CNPJ,RAZAO
0,1,10722640,"FERCOPY, INC"


In [151]:
base = ch.merge(df, left_on='ID',  right_on='Id')

In [152]:
headers = {'Authorization':'QGwuS9KGnXSt5s5A'}

url = "https://usagetracker.exactsales.com.br/api/v1/clientes/Empresas?pDtInicio=2010-01-01&pLayoutRetorno=Linear"
r = requests.get(url, headers=headers)
c1=pd.DataFrame(r.json())

url = "https://usagetracker.exactsales.com.br/api/v2/clientes/Empresas?pDtInicio=2010-01-01&pLayoutRetorno=Linear"
r = requests.get(url, headers=headers)
c2=pd.DataFrame(r.json())

cli = pd.concat([c1,c2])

In [153]:
cli.head(1)

,SpotterId,ID1,NomeEmpresa,RazaoSocial,StatusEmpresa,CadastroEmpresa,TelefoneEmpresa,LimiteUsuarios
0,4,78614229000103,Multilog,MULTILOG S.A.,Inativo,2015-05-03,None,5000


In [154]:
cli['ID1'] = cli['ID1'].astype('str')

In [155]:
cli.rename(columns={'ID1':'cnpj', 'RazaoSocial':'RAZAO'},inplace=True)

In [156]:
base = base.merge(cli[['RAZAO','cnpj','SpotterId']])

In [157]:
base.drop(columns=['ID','CNPJ','Id'], inplace=True)

In [158]:
base.tail()

,RAZAO,VlrNota,Dt. Evento,Ano Evento,Receita,Dt. Entrada,Tempo de vida da safra,Ano Entrada,Porte Ajustado,Origem ajustada,Integração RD Mkt e CRM,Webphone,Exact Club,Média de Leads em Entrada (Categorias),CNAE,cnpj,SpotterId
62571,BIN.COM PRODUTOS ALIMENTICIOS EIRELI - ME,"R$ 699,00",01/12/2018,2018,Spotter,01/11/2017,14,2017,SEM PORTE DEFINIDO,Lista ativa,False,False,False,-,Fabricação de outros produtos alimentícios não...,11226117000104,1027
62572,PHOCUS PROPAGANDA E MARKETING LTDA,"R$ 799,00",01/12/2018,2018,Spotter,01/08/2018,5,2018,SEM PORTE DEFINIDO,Lista ativa,False,False,False,-,Agências de publicidade,03698527000120,1418
62573,PORTAL ENGENHARIA LTDA,"R$ 699,00",01/12/2018,2018,Spotter,01/05/2018,8,2018,SEM PORTE DEFINIDO,Lista ativa,False,False,False,-,Serviços de engenharia,02363128000145,1256
62574,PRIME LASER INFORMATICA LTDA,"R$ 799,00",01/12/2018,2018,Spotter,01/09/2018,4,2018,SEM PORTE DEFINIDO,Lista ativa,False,False,False,-,Comércio atacadista de suprimentos para inform...,06886738000184,1520
62575,TETRALON IND E COM DE EQUIPAMENTOS INDUSTRIAIS...,"R$ 499,00",01/12/2018,2018,Spotter,01/05/2017,20,2017,SEM PORTE DEFINIDO,Lista ativa,False,False,False,-,Fabricação de outras máquinas e equipamentos d...,53945507000184,770


In [159]:
base.columns = ['razao_social', 'valor', 'data', 'ano', 'produto',
       'data_base', 'lt', 'ano_base', 'porte', 'origem', 'integracao',
       'webphone', 'club', 'leads', 'cnae', 'cnpj', 'SpotterId']

In [160]:
mrr = base[['cnpj', 'razao_social', 'data_base', 'SpotterId', 'data', 'produto', 'valor']]
cliente = base[['cnpj', 'razao_social', 'SpotterId', 'data_base', 'lt', 'ano_base', 'porte', 'origem']].drop_duplicates()

In [161]:
mrr.head(1)

,cnpj,razao_social,data_base,SpotterId,data,produto,valor
0,30500106316,LA LEY SOCIEDAD ANONIMA EDITORA E IMPRESORA,01/11/2019,4680,01/10/2022,Spotter,"R$ 2.444,00"


In [162]:
def to_float(x):

    x = x[2:]
    x = x.replace('.','')
    x = x.replace(',','.')
    x = float(x)
    return x

mrr.valor = mrr.valor.apply(to_float)

/home/vini/exact/exact/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [163]:
mrr.data = pd.to_datetime(mrr.data, format='%d/%m/%Y')
mrr.data_base = pd.to_datetime(mrr.data_base, format='%d/%m/%Y')

In [164]:
mrr.SpotterId = mrr.SpotterId.astype('str')

In [165]:
cliente.head(1)

,cnpj,razao_social,SpotterId,data_base,lt,ano_base,porte,origem
0,30500106316,LA LEY SOCIEDAD ANONIMA EDITORA E IMPRESORA,4680,01/11/2019,36,2019,Middle,Indicação


In [166]:
cliente.data_base = pd.to_datetime(cliente.data_base, format='%d/%m/%Y')

In [167]:
cliente.SpotterId = cliente.SpotterId.astype('str')

In [168]:
import dateutil.relativedelta as rd

In [169]:
mrr['meses'] = 0
for i in mrr.index:

    d = rd.relativedelta(mrr.data.loc[i], mrr.data_base.loc[i])
    mrr['meses'].loc[i] = d.months + (d.years * 12)

/tmp/ipykernel_736/1068792759.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/vini/exact/exact/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
mrr.describe()

,valor,meses
count,62576.000000,62576.000000
mean,533.557605,10.029628
std,634.753105,11.797192
min,11.500000,0.000000
25%,149.000000,2.000000
50%,398.000000,6.000000
75%,699.000000,13.000000
max,10218.000000,90.000000


In [171]:
mrr_lt = mrr.pivot_table(index='data_base',columns='meses', values='valor', aggfunc=sum)

In [172]:
for c in mrr_lt.columns:
    if c != 1:
        mrr_lt[c] = mrr_lt[c] / mrr_lt[1]

In [173]:
mrr_lt[1] = 1

In [174]:
mrr_lt = mrr_lt * 100

In [180]:

fig = px.imshow(mrr_lt.loc['2020':].iloc[:,:36], text_auto='.1f', color_continuous_scale='temps_r')
fig.show()

In [187]:
colors=['tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid', 'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr', 'ylorrd']
            

In [182]:
base.data_base = pd.to_datetime(base.data_base, format='%d/%m/%Y')

In [185]:
len(base.set_index('data_base').loc['2022-05':'2022-07'].razao_social.unique())

202

In [188]:
mrr.to_parquet('mrr.parquet')
cliente.to_parquet('cliente.parquet')